In [1]:
from eyemind.dataloading.load_dataset import *


/Users/rickgentry/miniforge3/envs/eyemind/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [23]:
def sequence_length_map(folder):
    seq_map = {}
    for f in Path(folder).glob("*.csv"):
        df = pd.read_csv(f)
        seq_len = len(df["XAvg"])
        seq_map[f.name] = seq_len
    return seq_map

def filter_by_min_seq(seq_map, min_seq_len=500):
    res = []
    count_filtered=0
    total_seq = 0
    for k, v in seq_map.items():
        if v > min_seq_len:
            res.append(k)
        else:
            count_filtered+=1
        total_seq+=1
    print(count_filtered)
    print(total_seq)
    return res

In [18]:
seq_map = sequence_length_map(Path("/Users/rickgentry/emotive_lab/eyemind/data/processed/fixation"))

In [25]:
filter_by_min_seq(seq_map,1000)

484
6279


['EML1_051-Hypotheses9.csv',
 'EML1_127-Validity8.csv',
 'EML1_075-Hypotheses8.csv',
 'EML1_005-Bias4.csv',
 'EML1_115-Bias6.csv',
 'EML1_072-CausalClaims4.csv',
 'EML1_017-Variables9.csv',
 'EML1_110-CausalClaims0.csv',
 'EML1_042-Bias9.csv',
 'EML1_111-CausalClaims6.csv',
 'EML1_120-Variables5.csv',
 'EML1_135-Variables5.csv',
 'EML1_053-Bias6.csv',
 'EML1_128-Variables4.csv',
 'EML1_104-Bias9.csv',
 'EML1_129-CausalClaims8.csv',
 'EML1_024-Variables6.csv',
 'EML1_038-Validity1.csv',
 'EML1_135-CausalClaims9.csv',
 'EML1_097-Bias5.csv',
 'EML1_031-Variables6.csv',
 'EML1_048-Hypotheses8.csv',
 'EML1_039-Variables7.csv',
 'EML1_060-Bias0.csv',
 'EML1_076-Validity6.csv',
 'EML1_126-Validity6.csv',
 'EML1_011-Validity5.csv',
 'EML1_021-CausalClaims3.csv',
 'EML1_019-Bias8.csv',
 'EML1_041-Variables2.csv',
 'EML1_085-Validity1.csv',
 'EML1_107-Hypotheses8.csv',
 'EML1_123-Hypotheses9.csv',
 'EML1_036-Bias2.csv',
 'EML1_126-Bias0.csv',
 'EML1_132-Validity3.csv',
 'EML1_062-Validity3.csv',